In [1]:
import pandas as pd
import os
import sys

In [2]:
# Set paths
homey = os.getcwd()

In [3]:
# Pull in most recent timeline
timeline = pd.read_excel(os.path.join(homey, 'RegularForecast.xlsx'), sheetname='Timeline')

In [53]:
# This will be a user input later
grandParent = '49977'

In [54]:
# Imaginary build datatypes end up as ints, so cast the column as str
timeline['GRANDPARENT'] = timeline['GRANDPARENT'].astype(str)

In [55]:
# Reduce to the order lines relating to the Grandparent
parentCheck = timeline[timeline['GRANDPARENT'].str.contains(grandParent, na=False)].copy()

In [56]:
# Isolate the purchasing needs for that Grandparent
purchaseForParent = parentCheck[parentCheck['ORDERTYPE'] == 'Purchase'].copy()

In [57]:
purchaseForParent

,ORDER,ITEM,ORDERTYPE,PART,QTYREMAINING,INV,DATESCHEDULED,PARENT,Make/Buy,GRANDPARENT,DESCRIPTION
3375,1212,Phantom,Purchase,561-015-10,8.0,8.0,2017-08-08,59,Buy,49977,Standoff Insulators; 5/8 thread; 2 5/8 height;...
3937,321,Phantom,Purchase,590-1823-10 r02,4.0,4.0,2017-08-08,59,Buy,49977,Retun Bar; 6in x 27in; Holes for .625in and 1i...
4485,541,Phantom,Purchase,590-2250-10 r02,1.0,3.0,2017-08-07,339,Buy,49977,Front Blanking Panel; Half BDFB
4496,659,Phantom,Purchase,590-2251-10 r02,1.0,1.0,2017-08-07,340,Buy,49977,Front Blanking Panel w-cutout; Right; Half BDFB
4506,958,Phantom,Purchase,590-2254-10 r02,1.0,1.0,2017-08-06,689,Buy,49977,Door; Advanced Controller; Right; Half BDFB
6054,1023,Phantom,Purchase,630-063-10,7.0,7.0,2017-08-06,762,Buy,49977,Scrw; mach 1/4-20 x 3/4 serrated hex; wash; zinc
6130,1122,Phantom,Purchase,630-092-10,24.0,24.0,2017-08-07,338,Buy,49977,Scrw; mach #8-32 x 3/8 flat m/s; phh; u/c; st...
6425,1219,Phantom,Purchase,630-161-10,16.0,16.0,2017-08-06,762,Buy,49977,Scrw; Mach; 3/8-16 x 2; 9/16 Hex C/S; 18-8 SS;
8484,962,Phantom,Purchase,634-056-10,4.0,4.0,2017-08-07,780,Buy,49977,Nut assy; 3/8 Tee Nut for wood pallet
8606,615,Phantom,Purchase,641-026-10,8.0,8.0,2017-08-07,334,Buy,49977,Bshg; snap Blk; .124 thk; 0.875 od; 0.687 id; ...


Now look for the total purchase needed for these individual parts with shortages:

In [58]:
# Start by making a purchasing timeline for phantom and imaginary builds
purchaseWhole = timeline[(timeline['ORDERTYPE'] == 'Purchase')].copy()
reducedPurchase = purchaseWhole[(purchaseWhole['ITEM'] == 'Imaginary') | (purchaseWhole['ITEM'] == 'Phantom')].copy()

In [59]:
# For each unique part you are checking, pull all the related order lines
totalShortParts = reducedPurchase[reducedPurchase['PART'].isin(purchaseForParent['PART'].unique().tolist())].copy()

In [60]:
# Sum the total orders needed for these parts
sumOrders = totalShortParts[['PART','QTYREMAINING']].groupby('PART').sum()
sumOrders.reset_index(inplace=True)
sumOrders.rename(columns={'QTYREMAINING':'TOTAL QTY'}, inplace=True)

In [61]:
# Merge it back onto the original list
sumOrders = pd.merge(purchaseForParent.copy(), sumOrders.copy(), how='left', on='PART')

In [62]:
# And polish for delivery
finalOutput = sumOrders[['ORDERTYPE','PART','DESCRIPTION','QTYREMAINING','TOTAL QTY','DATESCHEDULED','GRANDPARENT']].copy()
finalOutput.rename(columns={'QTYREMAINING':'ORDER QTY'}, inplace=True)

In [63]:
finalOutput

,ORDERTYPE,PART,DESCRIPTION,ORDER QTY,TOTAL QTY,DATESCHEDULED,GRANDPARENT
0,Purchase,561-015-10,Standoff Insulators; 5/8 thread; 2 5/8 height;...,8.0,94.0,2017-08-08,49977
1,Purchase,590-1823-10 r02,Retun Bar; 6in x 27in; Holes for .625in and 1i...,4.0,10.0,2017-08-08,49977
2,Purchase,590-2250-10 r02,Front Blanking Panel; Half BDFB,1.0,1.0,2017-08-07,49977
3,Purchase,590-2251-10 r02,Front Blanking Panel w-cutout; Right; Half BDFB,1.0,3.0,2017-08-07,49977
4,Purchase,590-2254-10 r02,Door; Advanced Controller; Right; Half BDFB,1.0,3.0,2017-08-06,49977
5,Purchase,630-063-10,Scrw; mach 1/4-20 x 3/4 serrated hex; wash; zinc,7.0,86.0,2017-08-06,49977
6,Purchase,630-092-10,Scrw; mach #8-32 x 3/8 flat m/s; phh; u/c; st...,24.0,192.0,2017-08-07,49977
7,Purchase,630-161-10,Scrw; Mach; 3/8-16 x 2; 9/16 Hex C/S; 18-8 SS;,16.0,313.0,2017-08-06,49977
8,Purchase,634-056-10,Nut assy; 3/8 Tee Nut for wood pallet,4.0,16.0,2017-08-07,49977
9,Purchase,641-026-10,Bshg; snap Blk; .124 thk; 0.875 od; 0.687 id; ...,8.0,11.0,2017-08-07,49977
